In [1]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sklearn as skl
import tensorflow as tf
import hvplot.pandas
import os

In [2]:
#loading csv file
path_to_file = os.path.join('Resources', 'charity_data.csv')
charity_df = pd.read_csv(path_to_file)


In [3]:
#examine dataset
display(charity_df.head())
display(charity_df.dtypes)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [4]:
#cehcking for duplicates if EIN, if no duplicates then we can use them later as index in data frame
duplicateEIN = charity_df['EIN'].duplicated().sum()
print(duplicateEIN)

0


In [5]:
#checking for missing data
for column in charity_df:
    null_count = charity_df[column].isnull().sum()
    print(f'{column} has {null_count} missing data' ) 

EIN has 0 missing data
NAME has 0 missing data
APPLICATION_TYPE has 0 missing data
AFFILIATION has 0 missing data
CLASSIFICATION has 0 missing data
USE_CASE has 0 missing data
ORGANIZATION has 0 missing data
STATUS has 0 missing data
INCOME_AMT has 0 missing data
SPECIAL_CONSIDERATIONS has 0 missing data
ASK_AMT has 0 missing data
IS_SUCCESSFUL has 0 missing data


In [6]:
charity_df['EIN']

0         10520599
1         10531628
2         10547893
3         10553066
4         10556103
           ...    
34294    996009318
34295    996010315
34296    996012607
34297    996015768
34298    996086871
Name: EIN, Length: 34299, dtype: int64

We take "is successfull" column as a Target - meausure of the organization performance 

In [7]:
#ceting filter on charity_df that finds obj. columns
charity_obj_filter = charity_df.dtypes[charity_df.dtypes == object].index.tolist()
display(charity_obj_filter)
#we remove name from the list because we dont need it for data encoding, also tere is EIN
#i think i will use it as index
charity_obj_filter.remove('NAME')
display(charity_obj_filter)

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [8]:
#checking if bucketting is necessary
charity_df[charity_obj_filter].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

cleanning CLASSIFICATION column

In [9]:
classification_count = charity_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2570        1
C3700        1
C1728        1
C4200        1
C1820        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
classification_count.hvplot.density()

:Distribution   [CLASSIFICATION]   (Density)

In [11]:
classification_count.hvplot.bar()

:Bars   [index]   (CLASSIFICATION)

In [12]:
#we classify all classification with counts less than 150 becasue 190 is the last significant 
small_count_class_list = list(classification_count[classification_count<150].index)
#small_count_class

#we iterate over small_count_list and replace items in charity_df['CLASSIFICATION']
for small_class in small_count_class_list:
    charity_df['CLASSIFICATION'] = charity_df['CLASSIFICATION'].replace(small_class,'Other')

#display reorganized column
charity_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

cleaning APPLICATION_TYPE column

In [13]:
application_type_count = charity_df['APPLICATION_TYPE'].value_counts()
application_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [14]:
#we classify all application with counts less than 150  
small_application_list = list(application_type_count[application_type_count<150].index)


#we iterate over small_count_list and replace items in charity_df['CLASSIFICATION']
for small_application in small_application_list:
    charity_df['APPLICATION_TYPE'] = charity_df['APPLICATION_TYPE'].replace(small_application,'Other')

#display reorganized column
charity_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [15]:
#using one hot encoder co convert object columns into numbers
enc = OneHotEncoder(sparse=False)

#transforming filtered dataframe and saving it as encode_df
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_obj_filter]))

#impporting df column names
encode_df.columns = enc.get_feature_names(charity_obj_filter)

#checking the result
display(encode_df.head(2))

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [16]:
#merge encoded columns
charity_df=charity_df.merge(encode_df,left_index=True, right_index=True)

#removing not encoded columns
charity_df = charity_df.drop(charity_obj_filter,1)

#display prepared data frame
charity_df

,EIN,NAME,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,1,5000,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,1,108590,1,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,1,5000,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,1,6692,1,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,1,142590,1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,1,5000,0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,1,5000,0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,996012607,PTA HAWAII CONGRESS,1,5000,0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,1,5000,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
#creating df with organization name as index
icharity_df = charity_df.copy()
icharity_df = icharity_df.set_index('NAME')
icharity_df = icharity_df.drop('EIN',1)
icharity_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
NAME,,,,,,,,,,,,,,,,,,,,,
BLUE KNIGHTS MOTORCYCLE CLUB,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
AMERICAN CHESAPEAKE CLUB CHARITABLE TR,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ST CLOUD PROFESSIONAL FIREFIGHTERS,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
SOUTHSIDE ATHLETIC ASSOCIATION,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
GENETIC RESEARCH INSTITUTE OF THE DESERT,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
THE LIONS CLUB OF HONOLULU KAMEHAMEHA,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
INTERNATIONAL ASSOCIATION OF LIONS CLUBS,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
PTA HAWAII CONGRESS,1,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Data pre processing

In [18]:
#splitting data into train test
y = icharity_df['IS_SUCCESSFUL'].values
X = icharity_df.drop('IS_SUCCESSFUL',1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [19]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)


# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_scaled = X_scaler.transform(X)

display(X_train[0])
print(len(X_train[0]))

array([1.e+00, 5.e+03, 0.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00,
       1.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00])

47


imaging the data

In [20]:
from sklearn.decomposition import PCA
#we define dimension reductor, and set 2 as the number of dimension we want to keep
pca = PCA(n_components=2)
#we apply the reductor on the data set 
X_pca = pca.fit_transform(X_scaled)

In [21]:
X_pca_df = pd.DataFrame(X_pca, columns = ['PC1', 'PC2'] )
X_pca_df['Target'] = y
display(X_pca_df)

,PC1,PC2,Target
0,3.306467,0.964384,1
1,-1.149519,2.041966,1
2,6.166018,0.625954,0
3,-0.959952,-0.396056,1
4,-1.258790,3.516968,1
...,...,...,...
34294,3.777050,1.967353,0
34295,6.017760,0.567034,0
34296,0.892484,-2.484819,0
34297,5.171717,2.142024,1


In [22]:
X_pca_df.hvplot.scatter(x="PC1",y="PC2", hover_cols=["Target"], by="Target")

:NdOverlay   [Target]
   :Scatter   [PC1]   (PC2,Target)

Building deep learning model

In [23]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 32
hidden_nodes_layer4 = 16
hidden_nodes_layer5 = 8
hidden_nodes_layer6 = 4
hidden_nodes_layer7 = 2

neuro_model = tf.keras.models.Sequential()

# 1st hidden layer
neuro_model.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# 2nd hidden layer
neuro_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# 3rd hidden layer
neuro_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# 4rd hidden layer
neuro_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# 6th hidden layer
neuro_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# 7th hidden layer
neuro_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="relu"))

# Output layer
neuro_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
neuro_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               6144      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5

In [24]:
# Compile the Sequential model together and customize metrics
neuro_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Fit the model to the training data
fit_model = neuro_model.fit(X_train_scaled, y_train, epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 77us/sample - loss: 0.5681 - accuracy: 0.7201
Epoch 2/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5538 - accuracy: 0.7289
Epoch 3/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5507 - accuracy: 0.7309
Epoch 4/100
25724/25724 [==============================] - 2s 84us/sample - loss: 0.5483 - accuracy: 0.7322
Epoch 5/100
25724/25724 [==============================] - 2s 66us/sample - loss: 0.5477 - accuracy: 0.7325
Epoch 6/100
25724/25724 [==============================] - 2s 70us/sample - loss: 0.5467 - accuracy: 0.7337
Epoch 7/100
25724/25724 [==============================] - 2s 65us/sample - loss: 0.5452 - accuracy: 0.7338
Epoch 8/100
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5446 - accuracy: 0.7347
Epoch 9/100
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5439 - accuracy: 0.7352
Epoch

25724/25724 [==============================] - 1s 45us/sample - loss: 0.5299 - accuracy: 0.7418
Epoch 77/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5306 - accuracy: 0.7417
Epoch 78/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5298 - accuracy: 0.7411
Epoch 79/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5295 - accuracy: 0.7409
Epoch 80/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5296 - accuracy: 0.7414
Epoch 81/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5295 - accuracy: 0.7416
Epoch 82/100
25724/25724 [==============================] - 1s 41us/sample - loss: 0.5293 - accuracy: 0.7417
Epoch 83/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5300 - accuracy: 0.7410
Epoch 84/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5299 - accuracy: 0.7417
Epoch 85/100
25724/25724 [======

In [26]:
model_loss, model_accuracy = neuro_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5796 - accuracy: 0.7276
Loss: 0.5796236631474064, Accuracy: 0.727580189704895
